# model-1

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from skimage.metrics import structural_similarity as ssim_func
from skimage.metrics import peak_signal_noise_ratio as psnr_func
from skimage.metrics import mean_squared_error as mse_func

In [ ]:
# 1. LOAD DATA
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

In [ ]:
# 2. ADD RANDOM GAUSSIAN NOISE (Varying intensities)
def add_noise(images):
    noisy = []
    for img in images:
        # Reduced noise slightly to help CNN convergence initially
        sigma = np.random.uniform(0.2, 0.4) 
        noise = np.random.normal(0, sigma, img.shape)
        noisy.append(np.clip(img + noise, 0, 1))
    return np.array(noisy)

x_train_noisy = add_noise(x_train)
x_test_noisy = add_noise(x_test)

In [ ]:
# 3. CNN AUTOENCODER (Improved Architecture)
input_img = layers.Input(shape=(28, 28, 1))
# Encoder
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = models.Model(input_img, decoded)
# Using binary_crossentropy helps prevent the "all black" output on MNIST
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Increase epochs to at least 10 for meaningful patterns
autoencoder.fit(x_train_noisy, x_train, epochs=10, batch_size=128, shuffle=True, verbose=1)

In [ ]:
def spatial_wiener_filter(noisy_img, noise_variance, block_size=5):

    # Convert to 2D grayscale
    noisy_img = noisy_img.squeeze().astype(np.float64)

    kernel = (block_size, block_size)
    local_mean = cv2.blur(noisy_img, kernel)

    local_mean_sq = cv2.blur(noisy_img ** 2, kernel)
    local_variance = local_mean_sq - (local_mean ** 2)
    local_variance = np.maximum(local_variance, 0)

    signal_variance = np.maximum(0, local_variance - noise_variance)
    ratio = signal_variance / (local_variance + 1e-10)

    res = local_mean + ratio * (noisy_img - local_mean)

    # Return back in CNN format
    return np.clip(res, 0, 1).reshape(28, 28, 1)


In [ ]:
# 5. PREDICT AND COMPARE (Digits 0-9)
indices = [np.where(y_test == i)[0][0] for i in range(10)]
gt_samples = x_test[indices]
noisy_samples = x_test_noisy[indices]
cnn_results = autoencoder.predict(noisy_samples)
math_results = np.array([spatial_wiener_filter(img,0.0625,block_size=5)for img in noisy_samples])

In [ ]:
# --- Updated Step 2: Track noise parameters during generation ---
def add_noise_with_params(images):
    noisy_list = []
    params_list = []
    for img in images:
        mean = 0.0  # Common for Gaussian noise
        sigma = np.random.uniform(0.15, 0.4) 
        noise = np.random.normal(mean, sigma, img.shape)
        noisy_img = np.clip(img + noise, 0, 1)
        
        noisy_list.append(noisy_img)
        params_list.append((mean, sigma))
    return np.array(noisy_list), params_list

# Generate noisy test samples and store their specific noise levels
x_test_noisy, noise_params = add_noise_with_params(x_test)

# --- Updated Step 6: Enhanced Plot with Dynamic Labels ---
fig, axes = plt.subplots(4, 10, figsize=(22, 12))
plt.subplots_adjust(wspace=0.4, hspace=0.6)

labels = ["Original", "Noisy\n(Actual Noise)", "CNN Denoise\n(Metrics)", "Math Denoise\n(Metrics)"]

for i in range(10):
    # Get the noise parameters for the current digit column
    m, s = noise_params[indices[i]]
    
    imgs = [gt_samples[i], noisy_samples[i], cnn_results[i], math_results[i]]
    
    for r in range(4):
        ax = axes[r, i]
        ax.imshow(imgs[r].squeeze(), cmap='gray')
        ax.axis('off')
        
        # Row Labels (Only on the first column)
        if i == 0:
            ax.set_ylabel(labels[r], rotation=0, labelpad=50, fontsize=11, fontweight='bold', va='center')
            ax.axis('on') # Turn on to show ylabel
            ax.set_xticks([]); ax.set_yticks([]) # But hide ticks

        # Label Row 1: The specific noise applied
        if r == 1:
            ax.set_title(f"µ={m:.2f}, σ={s:.2f}", fontsize=9, color='red')

        # Label Rows 2 & 3: Comparison Metrics
        if r >= 2:
            psnr_v = psnr_func(gt_samples[i].squeeze(), imgs[r].squeeze(), data_range=1.0)
            mse_v = mse_func(gt_samples[i].squeeze(), imgs[r].squeeze())
            ssim_v = ssim_func(gt_samples[i].squeeze(), imgs[r].squeeze(), data_range=1.0)
            
            # Using color coding: Green for good, Black for standard
            color = 'green' if r == 2 else 'blue'
            ax.set_title(f"PSNR: {psnr_v:.1f}\nMSE: {mse_v:.3f}\nSSIM: {ssim_v:.2f}", 
                         fontsize=9, color=color, fontweight='bold')

plt.suptitle("Comparison Matrix: Stochastic Gaussian Noise Removal", fontsize=18, y=0.98)
plt.show()

In [ ]:
# --- Metric storage ---
cnn_psnr, cnn_mse, cnn_ssim = [], [], []
math_psnr, math_mse, math_ssim = [], [], []

for i in range(10):
    gt = gt_samples[i].squeeze()
    
    cnn_img = cnn_results[i].squeeze()
    math_img = math_results[i].squeeze()
    
    # CNN metrics
    cnn_psnr.append(psnr_func(gt, cnn_img, data_range=1.0))
    cnn_mse.append(mse_func(gt, cnn_img))
    cnn_ssim.append(ssim_func(gt, cnn_img, data_range=1.0))
    
    # Math metrics
    math_psnr.append(psnr_func(gt, math_img, data_range=1.0))
    math_mse.append(mse_func(gt, math_img))
    math_ssim.append(ssim_func(gt, math_img, data_range=1.0))
avg_cnn = (
    np.mean(cnn_psnr),
    np.mean(cnn_mse),
    np.mean(cnn_ssim)
)

avg_math = (
    np.mean(math_psnr),
    np.mean(math_mse),
    np.mean(math_ssim)
)
fig, axes = plt.subplots(4, 10, figsize=(18, 8))

rows = [
    gt_samples,
    noisy_samples,
    cnn_results,
    math_results
]

row_titles = [
    "Original",
    "Noisy",
    "CNN Denoised",
    "Wiener Denoised"
]

for r in range(4):
    for c in range(10):
        axes[r, c].imshow(rows[r][c].squeeze(), cmap='gray')
        axes[r, c].axis('off')
        
        if c == 0:
            axes[r, c].set_ylabel(
                row_titles[r],
                fontsize=12,
                rotation=90,
                labelpad=15
            )
fig.text(
    0.5, 0.02,
    f"CNN →  Avg PSNR: {avg_cnn[0]:.2f} | "
    f"Avg MSE: {avg_cnn[1]:.4f} | "
    f"Avg SSIM: {avg_cnn[2]:.3f}\n"
    f"Wiener → Avg PSNR: {avg_math[0]:.2f} | "
    f"Avg MSE: {avg_math[1]:.4f} | "
    f"Avg SSIM: {avg_math[2]:.3f}",
    ha='center',
    fontsize=12,
    fontweight='bold'
)

plt.suptitle(
    "Denoising Comparison with Average Evaluation Metrics",
    fontsize=16
)

plt.show()
